In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import ipywidgets as widgets
import sys
reload(sys)
sys.setdefaultencoding('utf-8')


NameError: name 'reload' is not defined

In [2]:
turmas = pd.read_csv('Dados Turma/turmas-2016.2.csv', sep=';')
disciplinas = pd.read_csv('componentes-curriculares-presenciais.csv', 
                          sep=';', 
                          usecols=["id_componente","codigo","nivel","nome","unidade_responsavel"])
docentes = pd.read_csv('docentes.csv', sep=';')



In [3]:
turmas.rename(columns={'id_componente_curricular': 'id_componente'}, inplace=True)
turmas.id_componente = turmas.id_componente.astype(str)

# Limpando o dataframe disciplinas com linhas desnecessárias
disciplinas.dropna(inplace=True)
disciplinas = disciplinas[~disciplinas.id_componente.duplicated()]




In [5]:
colunas = ["id_turma","id_docente_interno","codigo","nivel_ensino","nome","unidade_responsavel",
          "situacao_turma","capacidade_aluno","descricao_horario"]

merge_turmas_disciplinas = pd.merge(turmas, disciplinas, on='id_componente')[colunas]
merge_turmas_disciplinas.dropna(inplace=True)
merge_turmas_disciplinas.head()

,id_turma,id_docente_interno,codigo,nivel_ensino,nome,unidade_responsavel,situacao_turma,capacidade_aluno,descricao_horario
0,57581492,5753060.0,GPU0009,LATO SENSU,INOVAÇÃO NA GESTÃO DE POLÍTICAS PÚBLICAS,PROGRAMA DE PÓS-GRADUAÇÃO EM GESTÃO PÚBLICA,CONSOLIDADA,50.0,7M2345 35T23456 7T1
1,57583343,5752921.0,MUT311,TÉCNICO,APRECIAÇÃO MUSICAL II,ESCOLA DE MÚSICA,CONSOLIDADA,30.0,2T1234
2,57583344,5752917.0,MUT491,TÉCNICO,ARRANJOS II,ESCOLA DE MÚSICA,CONSOLIDADA,20.0,3T34
3,57583345,23400.0,MUT102,TÉCNICO,CANTO II,ESCOLA DE MÚSICA,CONSOLIDADA,3.0,3T123
4,57583346,35466.0,MUT102,TÉCNICO,CANTO II,ESCOLA DE MÚSICA,CONSOLIDADA,3.0,6T456


In [6]:
# filtrando as turmas com código IMD
turmas_imd = merge_turmas_disciplinas[merge_turmas_disciplinas.codigo.str.contains("IMD")]

# utilizar apenas disciplinas com código "CONSOLIDADA"
turmas_imd = turmas_imd[turmas_imd.situacao_turma == 'CONSOLIDADA']

# Adaptar o nome da coluna id_docente_interno para id_servidor
turmas_imd.rename(columns={'id_docente_interno': 'id_servidor'}, inplace=True)

turmas_imd.capacidade_aluno = turmas_imd.capacidade_aluno.astype(np.int64)

# Adaptar os tipos de dados em turmas_imd para o merge com o dataframe docentes
turmas_imd.id_servidor = turmas_imd.id_servidor.astype(np.int64)
turmas_imd.head()

,id_turma,id_servidor,codigo,nivel_ensino,nome,unidade_responsavel,situacao_turma,capacidade_aluno,descricao_horario
3564,57586177,62353,IMD0029,GRADUAÇÃO,ESTRUTURA DE DADOS BÁSICAS I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,60,35M12 (25/07/2016 - 17/12/2016)
3565,57586351,5757975,IMD0029,GRADUAÇÃO,ESTRUTURA DE DADOS BÁSICAS I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,60,35T12 (25/07/2016 - 17/12/2016)
3566,57586359,5757905,IMD0029,GRADUAÇÃO,ESTRUTURA DE DADOS BÁSICAS I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,60,35N12 (25/07/2016 - 17/12/2016)
3567,57586359,5757071,IMD0029,GRADUAÇÃO,ESTRUTURA DE DADOS BÁSICAS I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,60,35N12 (25/07/2016 - 17/12/2016)
3568,57586361,5757883,IMD0029,GRADUAÇÃO,ESTRUTURA DE DADOS BÁSICAS I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,40,35N34 (25/07/2016 - 17/12/2016)


In [7]:
# merge os dataframes  turmas_imd e docentes utilizando como base a coluna id_servidor

colunas = ["id_turma", "codigo", "nivel_ensino", "nome_y",
          "nome_x","capacidade_aluno", "descricao_horario"]
turmas_imd_nomesprof = pd.merge(turmas_imd, docentes, on='id_servidor')[colunas]
lista_prof = turmas_imd_nomesprof.nome_y.unique()



In [12]:
lista_prof.sort()
turmas_imd_nomesprof.head()

,id_turma,codigo,nivel_ensino,nome_y,nome_x,capacidade_aluno,descricao_horario
0,57586177,IMD0029,GRADUAÇÃO,SELAN RODRIGUES DOS SANTOS,ESTRUTURA DE DADOS BÁSICAS I,60,35M12 (25/07/2016 - 17/12/2016)
1,57586198,IMD0030,GRADUAÇÃO,SELAN RODRIGUES DOS SANTOS,LINGUAGEM DE PROGRAMAÇÃO I,30,35M34 (25/07/2016 - 17/12/2016)
2,57586359,IMD0029,GRADUAÇÃO,CESAR RENNO COSTA,ESTRUTURA DE DADOS BÁSICAS I,60,35N12 (25/07/2016 - 17/12/2016)
3,57586373,IMD0030,GRADUAÇÃO,CESAR RENNO COSTA,LINGUAGEM DE PROGRAMAÇÃO I,32,35N34 (25/07/2016 - 17/12/2016)
4,57586359,IMD0029,GRADUAÇÃO,JORGE ESTEFANO SANTANA DE SOUZA,ESTRUTURA DE DADOS BÁSICAS I,60,35N12 (25/07/2016 - 17/12/2016)


In [13]:
#Essa celula pega a avaliação dos professores do imd
avaliacao = pd.read_csv("avaliacaodocencia.csv", sep=';')
av_imd = avaliacao[avaliacao['nome_docente'].isin(lista_prof)]
av_imd.head()
#avaliacao2 = (avaliacao[(avaliacao.ano.astype(str).str.contains("2016")) | (avaliacao.ano.astype(str).str.contains("2017"))]

,id_docente,nome_docente,id_turma,ano,periodo,qtd_discentes,postura_profissional_media,postura_profissional_DP,atuacao_profissional_media,atuacao_profissional_DP,autoavaliacao_aluno_media,autoavaliacao_aluno_DP
12856,24853,ANNE MAGALY DE PAULA CANUTO,1208019,2013,2,33,9.43,0.91,9.27,1.21,8.64,2.16
12857,24853,ANNE MAGALY DE PAULA CANUTO,1211641,2013,2,12,9.42,0.97,8.72,1.80,7.90,2.55
12858,24853,ANNE MAGALY DE PAULA CANUTO,1217954,2014,1,19,9.23,1.81,7.63,2.94,7.24,3.38
12859,24853,ANNE MAGALY DE PAULA CANUTO,1228386,2014,2,17,9.50,0.83,7.67,2.71,8.31,2.50
12860,24853,ANNE MAGALY DE PAULA CANUTO,1239412,2015,1,25,9.25,1.27,8.78,1.52,7.87,2.54


In [14]:
# merge os dataframes  av_imd e turmas_imd utilizando como base a coluna id_turma

av_imd_nomesprof = pd.merge(av_imd, turmas_imd, on='id_turma')
av_imd_nomesprof.head()

,id_docente,nome_docente,id_turma,ano,periodo,qtd_discentes,postura_profissional_media,postura_profissional_DP,atuacao_profissional_media,atuacao_profissional_DP,autoavaliacao_aluno_media,autoavaliacao_aluno_DP,id_servidor,codigo,nivel_ensino,nome,unidade_responsavel,situacao_turma,capacidade_aluno,descricao_horario
0,24853,ANNE MAGALY DE PAULA CANUTO,57586394,2016,2,12,10.00,0.00,9.47,1.14,7.89,3.24,24853,IMD0033,GRADUAÇÃO,PROBABILIDADE,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,60,24M12 (25/07/2016 - 17/12/2016)
1,24949,ANDRE MAURICIO CUNHA CAMPOS,57586250,2016,2,16,9.46,0.83,9.05,1.48,8.47,2.13,24949,IMD0012.0,GRADUAÇÃO,INTRODUÇÃO ÀS TÉCNICAS DE PROGRAMAÇÃO,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,60,35M34 (25/07/2016 - 17/12/2016)
2,24949,ANDRE MAURICIO CUNHA CAMPOS,57586259,2016,2,32,9.25,1.49,8.16,2.83,8.66,2.36,24949,IMD0012.1,GRADUAÇÃO,PRÁTICAS DE TÉCNICAS DE PROGRAMAÇÃO,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,40,3M56 (25/07/2016 - 17/12/2016)
3,24949,ANDRE MAURICIO CUNHA CAMPOS,57586277,2016,2,16,9.50,0.80,8.75,1.86,8.09,2.79,24949,IMD0012.1,GRADUAÇÃO,PRÁTICAS DE TÉCNICAS DE PROGRAMAÇÃO,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,40,5M56 (25/07/2016 - 17/12/2016)
4,25868,UMBERTO SOUZA DA COSTA,57586366,2016,2,23,9.88,0.33,8.38,2.35,8.57,1.99,25868,IMD0030,GRADUAÇÃO,LINGUAGEM DE PROGRAMAÇÃO I,INSTITUTO METROPOLE DIGITAL,CONSOLIDADA,30,35T34 (25/07/2016 - 17/12/2016)


In [15]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual

def evolucao(x):
    dados_prof = av_imd_nomesprof.loc[av_imd_nomesprof['nome_docente'] == x]
    return (dados_prof)

interact(evolucao, x=lista_prof,);

interactive(children=(Dropdown(description='x', options=('ADJA FERREIRA DE ANDRADE', 'ALLAN DE MEDEIROS MARTINS', 'ALUIZIO FERREIRA DA ROCHA NETO', 'ALYSON MATHEUS DE CARVALHO SOUZA', 'ANDRE LUIZ DA SILVA SOLINO', 'ANDRE MAURICIO CUNHA CAMPOS', 'ANNE MAGALY DE PAULA CANUTO', 'ANTONIO IGOR SILVA DE OLIVEIRA', 'ANTONIO WALLACE ANTUNES SOARES', 'ATHANASIOS TSOUANAS', 'AUGUSTO JOSE VENANCIO NETO', 'BRUNO SANTANA DA SILVA', 'CARLOS AUGUSTO PROLO', 'CESAR RENNO COSTA', 'CHARLES ANDRYE GALVAO MADEIRA', 'DANILO CURVELO DE SOUZA', 'DENNYS LEITE MAIA', 'DIOMADSON RODRIGUES BELFORT', 'EDGARD DE FARIA CORREA', 'EDUARDO NOGUEIRA CUNHA', 'EIJI ADACHI MEDEIROS BARBOSA', 'EVERTON RANIELLY DE SOUSA CAVALCANTE', 'GIBEON SOARES DE AQUINO JUNIOR', 'GUSTAVO GIRAO BARRETO DA SILVA', 'IRIS LINHARES PIMENTA', 'ISAAC FRANCO FERNANDES', 'ISABEL DILLMANN NUNES', 'ITAMIR DE MORAIS BARROCA FILHO', 'IVANOVITCH MEDEIROS DANTAS DA SILVA', 'JOAO CARLOS XAVIER JUNIOR', 'JOAO MARCOS DE ALMEIDA', 'JORGE ESTEFANO SANTANA DE SOUZA', 'JULIO CESAR PAULINO DE MELO', 'LORENA AZEVEDO DE SOUSA', 'LOURENA KARIN DE MEDEIROS ROCHA', 'LUCELIO DANTAS DE AQUINO', 'MARCIA JACYNTHA NUNES RODRIGUES LUCENA', 'MONICA MAGALHAES PEREIRA', 'NELSON ION DE OLIVEIRA', 'RAFAELA HORACINA SILVA ROCHA SOARES', 'SAMYR SILVA BEZERRA JACOME', 'SELAN RODRIGUES DOS SANTOS', 'SILVIA MARIA DINIZ MONTEIRO MAIA', 'SILVIO COSTA SAMPAIO', 'UMBERTO RIVIECCIO', 'UMBERTO SOUZA DA COSTA', 'VICENTE ANGELO DE SOUSA JUNIOR', 'WELLINGTON SILVA DE SOUZA', 'WILLIAM BRENNO DOS SANTOS OLIVEIRA'), value='ADJA FERREIRA DE ANDRADE'), Output()), _dom_classes=('widget-interact',))

In [16]:
% matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np
def evolucaoGrafico(x):
    dados_prof = av_imd_nomesprof.loc[av_imd_nomesprof['nome_docente'] == x]
    plt.plot(dados_prof['nome'], dados_prof['postura_profissional_media'])
    plt.show()
    
interactive_plot = interact(evolucaoGrafico, x=lista_prof,)
interactive_plot









interactive(children=(Dropdown(description='x', options=('ADJA FERREIRA DE ANDRADE', 'ALLAN DE MEDEIROS MARTINS', 'ALUIZIO FERREIRA DA ROCHA NETO', 'ALYSON MATHEUS DE CARVALHO SOUZA', 'ANDRE LUIZ DA SILVA SOLINO', 'ANDRE MAURICIO CUNHA CAMPOS', 'ANNE MAGALY DE PAULA CANUTO', 'ANTONIO IGOR SILVA DE OLIVEIRA', 'ANTONIO WALLACE ANTUNES SOARES', 'ATHANASIOS TSOUANAS', 'AUGUSTO JOSE VENANCIO NETO', 'BRUNO SANTANA DA SILVA', 'CARLOS AUGUSTO PROLO', 'CESAR RENNO COSTA', 'CHARLES ANDRYE GALVAO MADEIRA', 'DANILO CURVELO DE SOUZA', 'DENNYS LEITE MAIA', 'DIOMADSON RODRIGUES BELFORT', 'EDGARD DE FARIA CORREA', 'EDUARDO NOGUEIRA CUNHA', 'EIJI ADACHI MEDEIROS BARBOSA', 'EVERTON RANIELLY DE SOUSA CAVALCANTE', 'GIBEON SOARES DE AQUINO JUNIOR', 'GUSTAVO GIRAO BARRETO DA SILVA', 'IRIS LINHARES PIMENTA', 'ISAAC FRANCO FERNANDES', 'ISABEL DILLMANN NUNES', 'ITAMIR DE MORAIS BARROCA FILHO', 'IVANOVITCH MEDEIROS DANTAS DA SILVA', 'JOAO CARLOS XAVIER JUNIOR', 'JOAO MARCOS DE ALMEIDA', 'JORGE ESTEFANO SANTANA DE SOUZA', 'JULIO CESAR PAULINO DE MELO', 'LORENA AZEVEDO DE SOUSA', 'LOURENA KARIN DE MEDEIROS ROCHA', 'LUCELIO DANTAS DE AQUINO', 'MARCIA JACYNTHA NUNES RODRIGUES LUCENA', 'MONICA MAGALHAES PEREIRA', 'NELSON ION DE OLIVEIRA', 'RAFAELA HORACINA SILVA ROCHA SOARES', 'SAMYR SILVA BEZERRA JACOME', 'SELAN RODRIGUES DOS SANTOS', 'SILVIA MARIA DINIZ MONTEIRO MAIA', 'SILVIO COSTA SAMPAIO', 'UMBERTO RIVIECCIO', 'UMBERTO SOUZA DA COSTA', 'VICENTE ANGELO DE SOUSA JUNIOR', 'WELLINGTON SILVA DE SOUZA', 'WILLIAM BRENNO DOS SANTOS OLIVEIRA'), value='ADJA FERREIRA DE ANDRADE'), Output()), _dom_classes=('widget-interact',))

<function __main__.evolucaoGrafico>